# Test Set Generator

In this tutorial, we'll explore the test set generation module in Ragas to create a synthetic test set for a Retrieval-Augmented Generation (RAG)-based question-answering bot

In [2]:
# Instalar dependencias si es necesario
# !pip install llama-index-readers-llamaparse ragas langchain openai python-dotenv
import os
import glob
import asyncio
import nest_asyncio
from pathlib import Path  # <-- ¡Importación crucial para el error!
from typing import List

# --- LlamaParse, LangChain, Ragas Imports ---
from llama_parse import LlamaParse
from langchain_community.document_loaders import DirectoryLoader
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
import openai
from ragas.testset.graph import KnowledgeGraph
from ragas.testset.graph import Node, NodeType
from ragas.testset.transforms import apply_transforms
from ragas.testset.transforms import HeadlinesExtractor, HeadlineSplitter, KeyphrasesExtractor
from ragas.testset.persona import Persona
from ragas.testset.synthesizers.single_hop.specific import (
    SingleHopSpecificQuerySynthesizer,
)
from ragas.testset import TestsetGenerator
from dotenv import load_dotenv


/home/david/uc_chile/proyecto_aplicado_1/rag/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 1. CARGA DE ENTORNO Y VALIDACIÓN (TU CÓDIGO)
# ==========================================================
nest_asyncio.apply()
load_dotenv()

# Configuración OpenAI
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("❌ No se encontró OPENAI_API_KEY en el .env")

# Configuración Qdrant
QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
COLLECTION_NAME = "metabolomics_agent_db" # Tu colección definida

if not QDRANT_URL:
    raise ValueError("❌ No se encontró QDRANT_URL en el .env")
if not QDRANT_API_KEY:
    raise ValueError("❌ No se encontró QDRANT_API_KEY en el .env")

# Configuración LlamaCloud
LLAMA_CLOUD_API_KEY = os.getenv("LLAMA_CLOUD_API_KEY")
if not LLAMA_CLOUD_API_KEY:
    raise ValueError("❌ No se encontró LLAMA_CLOUD_API_KEY en el .env")

print("✅ Credenciales validadas correctamente.")


# Directorio de salida: Esto crea la carpeta DENTRO de '../data/'
MD_OUTPUT_DIR = Path("../data/data_md_files") 

# Aplicar nest_asyncio para entornos como Jupyter/Colab/etc.
nest_asyncio.apply()

✅ Credenciales validadas correctamente.


In [4]:
async def parsear_pdf(file_path: Path, output_dir: Path):
    """Parsea un solo archivo PDF a Markdown y lo guarda."""
    file_name = file_path.name
    print(f"   > ⏳ Procesando: {file_name}")
    try:
        parser = LlamaParse(result_type="markdown", language="en")
        
        # Usando aload_data() para consistencia con tu entorno
        documents = await parser.aload_data(str(file_path)) 
        
        if documents:
            output_file_path = output_dir / f"{file_path.stem}.md"
            
            # *** CORRECCIÓN: Concatenar el texto de TODOS los documentos devueltos ***
            markdown_content = "\n\n".join([doc.text for doc in documents])
            
            with open(output_file_path, "w", encoding="utf-8") as f:
                f.write(markdown_content)
                
            print(f"   > ✅ Convertido (Páginas: {len(documents)}): {file_name} -> {output_file_path.name}")
        else:
            print(f"   > ⚠️ LlamaParse no pudo extraer contenido de: {file_name}")

    except Exception as e:
        print(f"   > ❌ ERROR al procesar {file_name}. Detalle: {e}")

async def main_ingest():
    """Busca PDFs en '../data/data_files/' y coordina su conversión a Markdown."""
    # RUTA DE BÚSQUEDA: ../data/data_files/
    ruta_data = os.path.join("..", "data", "data_files", "*.pdf")
    pdf_files = [Path(f) for f in glob.glob(ruta_data)]
    
    print(f"\n📂 Buscando archivos en: {ruta_data}")
    
    if not pdf_files:
        print("❌ No se encontraron PDFs en la carpeta '../data/data_files/'. Deteniendo el pipeline.")
        print(f"   Directorio actual: {os.getcwd()}")
        return None

    print(f"✅ Se encontraron {len(pdf_files)} archivos PDF: {[f.name for f in pdf_files]}")

    # Crear el directorio de salida (../data/data_md_files)
    MD_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    
    print(f"   Iniciando conversión de {len(pdf_files)} archivos (Secuencial)...")
    
    # Procesamiento secuencial (estable)
    for f in pdf_files:
        await parsear_pdf(f, MD_OUTPUT_DIR)
        
    print("\n✅ Conversión a Markdown finalizada.")
    
    return MD_OUTPUT_DIR

# Ejecutar la conversión
output_dir_path = asyncio.run(main_ingest())


📂 Buscando archivos en: ../data/data_files/*.pdf
✅ Se encontraron 1 archivos PDF: ['1-s2.0-S0022316622152399-main.pdf']
   Iniciando conversión de 1 archivos (Secuencial)...
   > ⏳ Procesando: 1-s2.0-S0022316622152399-main.pdf
Started parsing the file under job_id 03440fdb-ad42-4f34-8cc0-e706a7725357
   > ✅ Convertido (Páginas: 7): 1-s2.0-S0022316622152399-main.pdf -> 1-s2.0-S0022316622152399-main.md

✅ Conversión a Markdown finalizada.


In [7]:
if not output_dir_path:
    print("No se cargará el Test Set ya que no se encontraron documentos fuente.")
    exit()

# Cargar documentos Markdown
path = str(output_dir_path) 
print(f"\n📂 Cargando documentos desde: {path}")
# El DirectoryLoader lee el .md que LlamaParse creó
loader = DirectoryLoader(path, glob="**/*.md")
docs = loader.load()

if not docs:
    print("❌ Error de carga: No se encontraron documentos Markdown para procesar.")
    exit()
else:
    print(f"✅ Se cargaron {len(docs)} documentos.")

# Setup de LLMs y Embeddings
# NOTA: Asegúrate de que las claves de OpenAI se cargaron en la Celda 1.
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
openai_client = openai.OpenAI()
generator_embeddings = OpenAIEmbeddings(client=openai_client, model="text-embedding-3-small")  

## Create Knowledge Graph (Grafo de Conocimiento)
# Inicializamos el grafo con el contenido de los documentos.
print("\n⚙️ Creando Knowledge Graph base...")
kg = KnowledgeGraph()
for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
print(f"   > Knowledge Graph inicial creado con {len(kg.nodes)} nodos base.")


📂 Cargando documentos desde: ../data/data_md_files
✅ Se cargaron 1 documentos.

⚙️ Creando Knowledge Graph base...
   > Knowledge Graph inicial creado con 1 nodos base.


/tmp/ipykernel_10294/1072707839.py:20: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))


In [8]:
## Setup the transforms
# Aplicamos transforms para extraer titulares, dividir contenido y obtener frases clave.
print("\n🛠️ Aplicando Transforms para enriquecer el Knowledge Graph...")
headline_extractor = HeadlinesExtractor(llm=generator_llm, max_num=20)
headline_splitter = HeadlineSplitter(max_tokens=1500)
keyphrase_extractor = KeyphrasesExtractor(llm=generator_llm)

transforms = [
    headline_extractor,
    headline_splitter,
    keyphrase_extractor
]

apply_transforms(kg, transforms=transforms)
print("   > Transforms aplicados (Headlines, Keyphrases).")

## Configuring Personas for Query Generation (Adaptadas a Bio-Actives)
print("\n👥 Definiendo Personas para generar diversidad de consultas:")

persona_first_time_analyst = Persona(
    name="First Time Analyst (Principiante)",
    role_description="Analista recién integrado al lab. Necesita guía clara sobre la identificación básica de metabolitos y la interpretación de datos LC-MS (m/z y RT).",
)

persona_experienced_chemist = Persona(
    name="Experienced Chemist (Experto)",
    role_description="Químico con experiencia buscando detalles finos. Interesado en isómeros, estructuras complejas, rutas biosintéticas y resultados internos de estudios anteriores.",
)

persona_bioactivity_researcher = Persona(
    name="Bioactivity Researcher (Biólogo)",
    role_description="Investigador enfocado en la función. Su prioridad es conocer las actividades biológicas, los ensayos in vitro/in vivo asociados y la toxicidad potencial de un compuesto.",
)

personas = [persona_first_time_analyst, persona_experienced_chemist, persona_bioactivity_researcher]   
print("   > Personas definidas.")


🛠️ Aplicando Transforms para enriquecer el Knowledge Graph...


Applying KeyphrasesExtractor: 100%|██████████| 16/16 [00:15<00:00,  1.01it/s]

   > Transforms aplicados (Headlines, Keyphrases).

👥 Definiendo Personas para generar diversidad de consultas:
   > Personas definidas.


In [9]:
import os
import pandas as pd
from pathlib import Path

# --- 5.1 Configuración de la Generación ---

## Query Generation Using Synthesizers
# Se define la distribución de consultas (50% titulares, 50% frases clave).
query_distribution = [
    (
        SingleHopSpecificQuerySynthesizer(llm=generator_llm, property_name="headlines"),
        0.5,
    ),
    (
        SingleHopSpecificQuerySynthesizer(
            llm=generator_llm, property_name="keyphrases"
        ),
        0.5,
    ),
]    
print("\n🎯 Synthesizers configurados para un mix de preguntas.")

# Inicializamos el generador con los LLMs, Embeddings, Knowledge Graph y Personas.
generator = TestsetGenerator(
    llm=generator_llm,
    embedding_model=generator_embeddings,
    knowledge_graph=kg,
    persona_list=personas,
)   

print("✅ Generador de Test Set inicializado. Listo para generar.")


🎯 Synthesizers configurados para un mix de preguntas.
✅ Generador de Test Set inicializado. Listo para generar.


In [12]:
# --- 5.2 Generación y Guardado ---

print("\n🚀 Iniciando la generación del Test Set (10 preguntas)...")

# Ejecución de la generación
testset = generator.generate(testset_size=10, query_distribution=query_distribution)

print("\n✅ Generación de Test Set completada.")

# --- 6. Guardar Resultados en evals/datasets/ ---
# 1. Definir la carpeta de salida dentro de evals/
OUTPUT_DIR = Path("datasets")
OUTPUT_DIR.mkdir(exist_ok=True) # Crea la carpeta si no existe

# 2. Extraer el nombre base del documento fuente (usando el nombre del archivo de origen)
base_filename = Path(docs[0].metadata["source"]).stem

# 3. Construir el nombre del archivo final
OUTPUT_FILENAME = OUTPUT_DIR / f"{base_filename}_testset.csv"

# Convertir a DataFrame y guardar
testset_df = testset.to_pandas()
testset_df.to_csv(OUTPUT_FILENAME, index=False)
print(f"💾 Test Set guardado exitosamente en: {os.path.abspath(OUTPUT_FILENAME)}")




🚀 Iniciando la generación del Test Set (10 preguntas)...


Generating Samples: 100%|██████████| 10/10 [00:10<00:00,  1.10s/it]



✅ Generación de Test Set completada.
💾 Test Set guardado exitosamente en: /home/david/uc_chile/proyecto_aplicado_1/rag/evals/datasets/1-s2.0-S0022316622152399-main_testset.csv


In [13]:
testset.to_pandas()

,user_input,reference_contexts,reference,persona_name,query_style,query_length,synthesizer_name
0,What is the significance of sample preparation...,[Human Nutrition and Metabolism\n\nAbsorption ...,Sample preparation is crucial in the study of ...,Bioactivity Researcher (Biólogo),PERFECT_GRAMMAR,MEDIUM,single_hop_specific_query_synthesizer
1,How do chemicals and materials play a role in ...,[Human Nutrition and Metabolism\n\nAbsorption ...,"In the study, chemicals and materials such as ...",First Time Analyst (Principiante),PERFECT_GRAMMAR,LONG,single_hop_specific_query_synthesizer
2,What is Blueberry study about?,[Human Nutrition and Metabolism\n\nAbsorption ...,The Blueberry study investigates the absorptio...,First Time Analyst (Principiante),POOR_GRAMMAR,SHORT,single_hop_specific_query_synthesizer
3,Wht is the absorption and metabolism of anthoc...,[Human Nutrition and Metabolism\n\nAbsorption ...,The absorption and metabolism of anthocyanins ...,Experienced Chemist (Experto),MISSPELLED,LONG,single_hop_specific_query_synthesizer
4,What are the key components of the MATERIALS A...,[Human Nutrition and Metabolism\n\nAbsorption ...,The MATERIALS AND METHODS section details the ...,Bioactivity Researcher (Biólogo),WEB_SEARCH_LIKE,MEDIUM,single_hop_specific_query_synthesizer
5,What is the focus of the study on anthocyanins...,[Human Nutrition and Metabolism\n\n Absorption...,The study focuses on the absorption and metabo...,Experienced Chemist (Experto),WEB_SEARCH_LIKE,MEDIUM,single_hop_specific_query_synthesizer
6,What happen with elderberry extract in study?,[ABSTRACT\n\nThe absorption and metabolism of ...,"In the study, four elderly women were given 12...",Bioactivity Researcher (Biólogo),POOR_GRAMMAR,SHORT,single_hop_specific_query_synthesizer
7,How do the antioxidant capacities of anthocyan...,[KEY WORDS:\n\nanthocyanin\n\nmetabolite\n\nel...,The antioxidant capacities of anthocyanins (AC...,First Time Analyst (Principiante),PERFECT_GRAMMAR,LONG,single_hop_specific_query_synthesizer
8,Que es peonidin y de donde viene en los metodo...,[MATERIALS AND METHODS\n\n Chemicals and mater...,Peonidin es uno de los seis compuestos que se ...,First Time Analyst (Principiante),POOR_GRAMMAR,LONG,single_hop_specific_query_synthesizer
9,What was the dosage of elderberry extract cons...,[Subjects and study design. The study protocol...,The study participants consumed 12 g of elderb...,Bioactivity Researcher (Biólogo),PERFECT_GRAMMAR,SHORT,single_hop_specific_query_synthesizer


### Imports y Definición de Clases RAG/Retriever

In [16]:
# --- Imports para RAG y Evaluación ---
from typing import Any, Dict, Optional
import os
import asyncio
from pathlib import Path
import pandas as pd
from openai import AsyncOpenAI
# Se asume que las librerías 'langchain_classic' fueron instaladas con otras dependencias
from langchain_classic.docstore.document import Document
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever as LangchainBM25Retriever
from langchain_community.document_loaders import DirectoryLoader
from ragas.metrics import DiscreteMetric
from ragas import experiment
from ragas import Dataset

# --- 1. BM25 Retriever para Documentos Locales ---
class BM25Retriever:
    """Retriever simple basado en BM25 para buscar en tu documento Markdown local."""
    
    def __init__(self, doc_path: Path, default_k=3):
        self.default_k = default_k
        self.retriever = self._build_retriever(doc_path)
    
    def _build_retriever(self, doc_path: Path) -> LangchainBM25Retriever:
        """Construye un retriever BM25 a partir de un archivo Markdown local."""
        print(f"Cargando documento desde: {doc_path}")
        
        # Cargamos el documento Markdown
        loader = DirectoryLoader(str(doc_path.parent), glob=doc_path.name)
        source_documents = loader.load()

        # División de documentos (Chunking)
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=100,
            add_start_index=True,
            strip_whitespace=True,
            separators=["\n\n", "\n", ".", " ", ""],
        )
        
        all_chunks = []
        for document in source_documents:
            chunks = text_splitter.split_documents([document])
            all_chunks.extend(chunks)
        
        # Deduplicación simple
        unique_chunks = []
        seen_content = set()
        for chunk in all_chunks:
            if chunk.page_content not in seen_content:
                seen_content.add(chunk.page_content)
                unique_chunks.append(chunk)
        
        print(f"Creados {len(unique_chunks)} fragmentos únicos para RAG.")
        
        # Se asume que 'rank_bm25' ya está instalado aquí.
        return LangchainBM25Retriever.from_documents(
            documents=unique_chunks,
            k=1,
        )
    
    def retrieve(self, query: str, top_k: int = None):
        """Recupera documentos para una consulta dada."""
        if top_k is None:
            top_k = self.default_k
        self.retriever.k = top_k
        return self.retriever.invoke(query)

# --- 2. Sistema RAG Simple ---
class RAG:
    """Sistema RAG simple para recuperación de documentos y generación de respuestas."""

    def __init__(self, llm_client: AsyncOpenAI, retriever: BM25Retriever, system_prompt=None, model="gpt-4o-mini", default_k=3):
        self.llm_client = llm_client
        self.retriever = retriever
        self.model = model
        self.default_k = default_k
        self.system_prompt = system_prompt or "Responde únicamente basándote en los documentos proporcionados. Sé conciso.\n\nPregunta: {query}\nDocumentos:\n{context}\nRespuesta:"

    async def query(self, question: str, top_k: Optional[int] = None) -> Dict[str, Any]:
        """Consulta el sistema RAG."""
        if top_k is None:
            top_k = self.default_k

        return await self._naive_query(question, top_k)

    async def _naive_query(self, question: str, top_k: int) -> Dict[str, Any]:
        """Maneja el RAG ingenuo: recupera una vez, luego genera."""
        # 1. Recuperar documentos
        docs = self.retriever.retrieve(question, top_k)

        if not docs:
            return {"answer": "No se encontraron documentos relevantes.", "retrieved_documents": [], "num_retrieved": 0}

        # 2. Construir el contexto
        context = "\n\n".join([f"Documento {i}:\n{doc.page_content}" for i, doc in enumerate(docs, 1)])
        prompt = self.system_prompt.format(query=question, context=context)

        # 3. Generar respuesta usando OpenAI
        response = await self.llm_client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}]
        )

        return {
            "answer": response.choices[0].message.content.strip(),
            "retrieved_documents": [{"content": doc.page_content, "metadata": doc.metadata, "document_id": i} for i, doc in enumerate(docs)],
            "num_retrieved": len(docs)
        }

### Inicialización del RAG y Carga del Test Set

In [17]:
# Inicialización
# NOTA: Asume que las variables de entorno para las claves API están definidas.

# 1. Inicializar el Cliente Async OpenAI
llm_client = AsyncOpenAI(api_key=os.environ["OPENAI_API_KEY"])

# 2. Definir la ruta al archivo Markdown del usuario (dentro de data/data_md_files/)
KNOWLEDGE_BASE_PATH = Path("../data/data_md_files/1-s2.0-S0022316622152399-main.md")

# 3. Construir el Retriever BM25 (Esta línea requirió 'rank_bm25')
bm25_retriever = BM25Retriever(doc_path=KNOWLEDGE_BASE_PATH)

# 4. Inicializar el Sistema RAG Simple
rag_system = RAG(llm_client=llm_client, retriever=bm25_retriever)

print("\n✅ Sistema RAG Inicializado.")
print("---")

# 5. Cargar el Test Set Ragas generado (desde evals/datasets/)
TESTSET_PATH = Path("datasets/1-s2.0-S0022316622152399-main_testset.csv")
if not TESTSET_PATH.exists():
    raise FileNotFoundError(f"Test set no encontrado en {TESTSET_PATH}. Asegúrate de que la Celda 5.2 fue ejecutada correctamente.")

eval_df = pd.read_csv(TESTSET_PATH)
print(f"✅ Cargado el dataset de evaluación con {len(eval_df)} muestras.")

Cargando documento desde: ../data/data_md_files/1-s2.0-S0022316622152399-main.md
Creados 45 fragmentos únicos para RAG.

✅ Sistema RAG Inicializado.
---
✅ Cargado el dataset de evaluación con 10 muestras.


### Setup de Métrica y Función de Evaluación

In [31]:
# Importaciones necesarias (asume Celda 6 ya ejecutada)
from ragas.metrics import DiscreteMetric
from ragas import experiment
from ragas import Dataset
from typing import Dict, Any

# --- 1. Definir Métrica de Corrección ---
# La métrica en sí misma no necesita cambios, solo las claves de acceso en la función.
correctness_metric = DiscreteMetric(
    name="correctness",
    prompt="""Compara la respuesta del modelo con la respuesta esperada y determina si es correcta.

Considera la respuesta correcta si:
1. Contiene la información clave de la respuesta esperada
2. Es precisa en base al contexto proporcionado
3. Responde adecuadamente a la pregunta

Retorna 'pass' si la respuesta es correcta, 'fail' si es incorrecta.

Pregunta: {question}
Respuesta Esperada: {expected_answer}
Respuesta del Modelo: {response}

Evaluación:""",
    allowed_values=["pass", "fail"],
)

# --- 2. Definir Función de Evaluación del Experimento (CORRECCIÓN FINAL DEFINITIVA) ---
@experiment()
async def evaluate_rag(row: Dict[str, Any], rag: RAG, llm) -> Dict[str, Any]:
    """
    Ejecuta la evaluación RAG en una sola fila del dataset.
    """
    # CORRECCIÓN FINAL: Usar los nombres de columna exactos de tu CSV
    question = row["user_input"] # <--- Columna de pregunta
    reference_answer = row["reference"] # <--- Columna de respuesta esperada

    # Consultar el sistema RAG
    rag_response = await rag.query(question, top_k=4)
    model_response = rag_response.get("answer", "")

    # Evaluar corrección asíncronamente
    score = await correctness_metric.ascore(
        question=question,
        expected_answer=reference_answer, # Usamos la respuesta de la columna 'reference'
        response=model_response,
        llm=llm
    )

    # Devolver los resultados de la evaluación
    result = {
        **row,
        "model_response": model_response,
        "correctness_score": score.value,
        "correctness_reason": score.reason,
        "retrieved_documents": [
            doc.get("content", "")[:200] + "..." if len(doc.get("content", "")) > 200 else doc.get("content", "")
            for doc in rag_response.get("retrieved_documents", [])
        ]
    }

    return result

print("✅ Función de Experimento corregida definitivamente para usar 'user_input' y 'reference'.")

✅ Función de Experimento corregida definitivamente para usar 'user_input' y 'reference'.


### Ejecución del Experimento RAG Inicial

In [33]:
# Importaciones necesarias
import asyncio
from datetime import datetime
from ragas.llms import llm_factory
from ragas import Dataset
import os
import pandas as pd 

async def run_evaluation():
    # --- 1. PREPARACIÓN DEL DATASET LOCAL ---
    print("\n⏳ Convirtiendo DataFrame a formato Ragas Dataset...")
    
    # Se usa eval_df (cargado del CSV local en Celda 7)
    ragas_dataset = Dataset.from_pandas(
        eval_df, 
        name="local_rag_testset", 
        backend="local/csv", 
        root_dir="." 
    )
    
    # --- 2. INICIALIZACIÓN DE COMPONENTES DE EVALUACIÓN ---
    llm_for_ragas = llm_factory('gpt-4o-mini', client=llm_client)

    # --- 3. EJECUTAR EL EXPERIMENTO ---
    exp_name = f"{datetime.now().strftime('%Y%m%d-%H%M%S')}_naive_rag_local"
    print(f"\n🚀 Iniciando experimento: {exp_name}")
    
    # Ejecución asíncrona usando evaluate_rag.arun
    results = await evaluate_rag.arun(
        ragas_dataset, 
        name=exp_name,
        rag=rag_system, 
        llm=llm_for_ragas 
    )

    # --- 4. IMPRIMIR RESULTADOS ---
    if results:
        pass_count = sum(1 for result in results if result.get("correctness_score") == "pass")
        total_count = len(results)
        pass_rate = (pass_count / total_count) * 100 if total_count > 0 else 0
        print(f"\n--- Resumen de Evaluación RAG ({exp_name}) ---")
        print(f"Resultados: {pass_count}/{total_count} pasaron ({pass_rate:.1f}%)")
        print("-" * 30)

    return results

# Ejecutar la evaluación 
evaluation_results = await run_evaluation()

# Mostrar resultados detallados
#print("\nResultados detallados:")
#pd.DataFrame(evaluation_results)


⏳ Convirtiendo DataFrame a formato Ragas Dataset...

🚀 Iniciando experimento: 20251126-164933_naive_rag_local


Running experiment: 100%|██████████| 10/10 [00:11<00:00,  1.10s/it]


--- Resumen de Evaluación RAG (20251126-164933_naive_rag_local) ---
Resultados: 4/10 pasaron (40.0%)
------------------------------


In [35]:
print("\nResultados detallados:")
#pd.DataFrame(evaluation_results)


Resultados detallados:


### Implementación del RAG Agéntico

In [39]:
# Importaciones necesarias para RAG Agéntico
from typing import Any, Dict, Optional
from openai import AsyncOpenAI 
import os
# ATENCIÓN: Se asume que las clases RAG, BM25Retriever ya están definidas
# ATENCIÓN: Se asume que has instalado la librería 'agents'
try:
    from agents import Agent, function_tool, Runner 
except ImportError:
    print("Error: El paquete 'agents' es requerido para el modo agéntico.")

class ImprovedRAG(RAG):
    """RAG system that can operate in naive or agentic mode."""

    def __init__(self, llm_client: AsyncOpenAI, retriever: BM25Retriever, mode="agentic", system_prompt=None, model="gpt-4o-mini", default_k=3):
        # MODIFICACIÓN PUNTUAL: Cambiado gpt-5-mini a gpt-4o-mini
        super().__init__(
            llm_client=llm_client,
            retriever=retriever,
            system_prompt=system_prompt,
            model=model,
            default_k=default_k
        )
        self.mode = mode.lower()
        self._agent = None
        
        if self.mode == "agentic":
            self._setup_agent()

    def _setup_agent(self):
        """Setup agent for agentic mode."""
        
        @function_tool
        def retrieve(query: str) -> str:
            """Search Hugging Face docs for technical info, APIs, commands, and examples.
            Use exact terms (e.g., "from_pretrained", "ESPnet upload", "torchrun"). 
            Try 2-3 targeted searches: specific terms → tool names → alternatives."""
            # self.retriever.retrieve usa el BM25 cargado con tu documento
            docs = self.retriever.retrieve(query, self.default_k)
            if not docs:
                return f"No documents found for '{query}'. Try different search terms or break down the query into smaller parts."
            return "\n\n".join([f"Doc {i}: {doc.page_content}" for i, doc in enumerate(docs, 1)])

        self._agent = Agent(
            name="RAG Assistant",
            model=self.model,
            instructions="Search with exact terms first (commands, APIs, tool names). Try 2-3 different searches if needed. Only answer from retrieved documents. Preserve exact syntax and technical details.",
            tools=[retrieve]
        )

    async def _agentic_query(self, question: str, top_k: int) -> Dict[str, Any]:
        """Handle agentic mode: agent controls retrieval strategy."""
        
        # Uso de la clase Runner del paquete 'agents'
        result = await Runner.run(self._agent, input=question)
        
        return {
            "answer": result.final_output,
            "retrieved_documents": [],  # Agent handles retrieval internally
            "num_retrieved": 0,         # Cannot determine exact count from agent execution
        }

    async def query(self, question: str, top_k: Optional[int] = None) -> Dict[str, Any]:
        """Query the RAG system."""
        if top_k is None:
            top_k = self.default_k
            
        try:
            if self.mode == "naive":
                return await self._naive_query(question, top_k)
            elif self.mode == "agentic":
                return await self._agentic_query(question, top_k)
            else:
                raise ValueError(f"Unknown mode: {self.mode}")
        except Exception as e:
            return {
                "answer": f"Error: {str(e)}", 
                "retrieved_documents": [], 
                "num_retrieved": 0,
            }
print("✅ Clase ImprovedRAG (Agéntica) definida.")

✅ Clase ImprovedRAG (Agéntica) definida.


### Prueba Rápida del RAG Agéntico

In [40]:
# Importaciones necesarias
from openai import AsyncOpenAI 
import os

# Inicializar cliente OpenAI (asumiendo que llm_client ya existe, pero por si acaso)
openai_client = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Inicializar el retriever BM25 (asumiendo que KNOWLEDGE_BASE_PATH existe de celdas previas)
retriever = BM25Retriever(doc_path=KNOWLEDGE_BASE_PATH)

# Switch to agentic mode
rag_agentic = ImprovedRAG(openai_client, retriever, mode="agentic")

question = "What architecture is the `tokenizers-linux-x64-musl` binary designed for?"
result = await rag_agentic.query(question)
print(f"Answer: {result['answer']}")

Cargando documento desde: ../data/data_md_files/1-s2.0-S0022316622152399-main.md
Creados 45 fragmentos únicos para RAG.
Answer: I couldn't find specific information on the architecture for the `tokenizers-linux-x64-musl` binary in the retrieved documents. However, based on the naming convention, it is likely designed for the x86_64 architecture (64-bit) on Linux systems that use the musl C library. If you need more details, I recommend checking the official documentation or repository related to the tokenizers library.


### Evaluación del RAG Agéntico

In [43]:
# Importaciones necesarias
import asyncio
from datetime import datetime
from ragas.llms import llm_factory
from ragas import Dataset
import os
import pandas as pd 

# La función evaluate_rag debe estar definida en la Celda 8 (con claves 'user_input' y 'reference')
# y la variable eval_df debe estar cargada (Celda 7)

async def run_agentic_evaluation():
    
    # --- 1. PREPARACIÓN DEL DATASET LOCAL (Reutilizando eval_df) ---
    print("\n⏳ Convirtiendo DataFrame a formato Ragas Dataset...")
    # Reutilizamos la lógica que funcionó
    ragas_dataset = Dataset.from_pandas(
        eval_df, 
        name="local_agentic_testset", 
        backend="local/csv", 
        root_dir="." 
    )
    
    # --- 2. INICIALIZACIÓN DE COMPONENTES DE EVALUACIÓN ---
    openai_client = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    retriever = BM25Retriever(doc_path=KNOWLEDGE_BASE_PATH)
    
    # Inicializar RAG Agéntico (MODIFICACIÓN PUNTUAL: modelo a gpt-4o-mini)
    rag = ImprovedRAG(llm_client=openai_client, retriever=retriever, model="gpt-4o-mini", mode="agentic")
    
    # Inicializar LLM para la evaluación de Ragas
    llm = llm_factory('gpt-4o-mini', client=openai_client)

    # --- 3. EJECUTAR EL EXPERIMENTO ---
    exp_name = f"{datetime.now().strftime('%Y%m%d-%H%M%S')}_agentic_rag"
    print(f"\n🚀 Iniciando experimento Agéntico: {exp_name}")
    
    # Ejecución de la evaluación
    results = await evaluate_rag.arun(
        ragas_dataset, 
        name=exp_name,
        rag=rag,
        llm=llm
    )

    # --- 4. IMPRIMIR RESULTADOS ---
    if results:
        pass_count = sum(1 for result in results if result.get("correctness_score") == "pass")
        total_count = len(results)
        pass_rate = (pass_count / total_count) * 100 if total_count > 0 else 0
        print(f"\n--- Resumen de Evaluación RAG Agéntico ({exp_name}) ---")
        print(f"Resultados: {pass_count}/{total_count} pasaron ({pass_rate:.1f}%)")
        print("-" * 55)

    return results

# Ejecutar la evaluación agéntica
results_agentic = await run_agentic_evaluation()

# Mostrar resultados detallados
#print("\nResultados detallados:")
#pd.DataFrame(results_agentic)


⏳ Convirtiendo DataFrame a formato Ragas Dataset...
Cargando documento desde: ../data/data_md_files/1-s2.0-S0022316622152399-main.md
Creados 45 fragmentos únicos para RAG.

🚀 Iniciando experimento Agéntico: 20251126-170027_agentic_rag


Running experiment: 100%|██████████| 10/10 [00:17<00:00,  1.71s/it]


--- Resumen de Evaluación RAG Agéntico (20251126-170027_agentic_rag) ---
Resultados: 5/10 pasaron (50.0%)
-------------------------------------------------------


In [44]:
# Mostrar resultados detallados
print("\nResultados detallados:")
pd.DataFrame(results_agentic)


Resultados detallados:


,user_input,reference_contexts,reference,persona_name,query_style,query_length,synthesizer_name,model_response,correctness_score,correctness_reason,retrieved_documents
0,What is Blueberry study about?,['Human Nutrition and Metabolism\n\nAbsorption...,The Blueberry study investigates the absorptio...,First Time Analyst (Principiante),POOR_GRAMMAR,SHORT,single_hop_specific_query_synthesizer,The Blueberry study refers to a research proje...,fail,The model's response does not accurately addre...,[]
1,What was the dosage of elderberry extract cons...,['Subjects and study design. The study protoco...,The study participants consumed 12 g of elderb...,Bioactivity Researcher (Biólogo),PERFECT_GRAMMAR,SHORT,single_hop_specific_query_synthesizer,The study participants consumed 12 grams of el...,pass,The model's response contains all the key info...,[]
2,Que es peonidin y de donde viene en los metodo...,['MATERIALS AND METHODS\n\n Chemicals and mate...,Peonidin es uno de los seis compuestos que se ...,First Time Analyst (Principiante),POOR_GRAMMAR,LONG,single_hop_specific_query_synthesizer,"**Peonidina** es un antociano, un tipo de pigm...",fail,La respuesta del modelo no menciona que la peo...,[]
3,What is the significance of sample preparation...,['Human Nutrition and Metabolism\n\nAbsorption...,Sample preparation is crucial in the study of ...,Bioactivity Researcher (Biólogo),PERFECT_GRAMMAR,MEDIUM,single_hop_specific_query_synthesizer,Sample preparation is a crucial step in the st...,fail,The model's response contains detailed informa...,[]
4,How do the antioxidant capacities of anthocyan...,['KEY WORDS:\n\nanthocyanin\n\nmetabolite\n\ne...,The antioxidant capacities of anthocyanins (AC...,First Time Analyst (Principiante),PERFECT_GRAMMAR,LONG,single_hop_specific_query_synthesizer,The antioxidant capacities of anthocyanins con...,pass,La respuesta del modelo contiene información c...,[]
5,What are the key components of the MATERIALS A...,['Human Nutrition and Metabolism\n\nAbsorption...,The MATERIALS AND METHODS section details the ...,Bioactivity Researcher (Biólogo),WEB_SEARCH_LIKE,MEDIUM,single_hop_specific_query_synthesizer,The **MATERIALS AND METHODS** section in the s...,pass,The model's response contains detailed informa...,[]
6,What is the focus of the study on anthocyanins...,['Human Nutrition and Metabolism\n\n Absorptio...,The study focuses on the absorption and metabo...,Experienced Chemist (Experto),WEB_SEARCH_LIKE,MEDIUM,single_hop_specific_query_synthesizer,The study on anthocyanins (ACN) focuses on the...,fail,The model response provides detailed informati...,[]
7,What happen with elderberry extract in study?,['ABSTRACT\n\nThe absorption and metabolism of...,"In the study, four elderly women were given 12...",Bioactivity Researcher (Biólogo),POOR_GRAMMAR,SHORT,single_hop_specific_query_synthesizer,"In studies involving elderberry extract, subje...",fail,The model's response contains key information ...,[]
8,Wht is the absorption and metabolism of anthoc...,['Human Nutrition and Metabolism\n\nAbsorption...,The absorption and metabolism of anthocyanins ...,Experienced Chemist (Experto),MISSPELLED,LONG,single_hop_specific_query_synthesizer,In a study examining the absorption and metabo...,pass,La respuesta del modelo contiene la informació...,[]
9,How do chemicals and materials play a role in ...,['Human Nutrition and Metabolism\n\nAbsorption...,"In the study, chemicals and materials such as ...",First Time Analyst (Principiante),PERFECT_GRAMMAR,LONG,single_hop_specific_query_synthesizer,In studies examining the absorption and metabo...,pass,The model's response contains detailed informa...,[]
